# Feature A Impact Analysis (Spain)

Observational analysis of Feature A launched in July 2022.

## 1. Setup & Data Loading

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler


In [ ]:

df = pd.read_csv("test_data_analyst_v2.csv")
df.head()


## 2. Data Preparation

In [ ]:

df["signup_date"] = pd.to_datetime(df["signup_date"])
df_esp = df[df["country_code"] == "ESP"].copy()

launch_date = pd.to_datetime("2022-07-01")
df_esp["post_launch"] = (df_esp["signup_date"] >= launch_date).astype(int)
df_esp.head()


## 3. Metric Engineering

In [ ]:

df_esp["engagement_depth"] = (
    df_esp["feature_A"] * 2 +
    df_esp["feature_B"] +
    df_esp["feature_C"] +
    df_esp["MAU"]
)

df_esp["PAU_plus"] = (
    (df_esp["MAU"] == 1) &
    ((df_esp["feature_A"] + df_esp["feature_B"] + df_esp["feature_C"]) >= 1)
).astype(int)


## 4. Matched Cohort Analysis

In [ ]:

df_esp["feature_A_user"] = (df_esp["feature_A"] == 1).astype(int)

matching_features = ["feature_B", "feature_C", "MAU", "PAU"]
X = df_esp[matching_features]
y = df_esp["feature_A_user"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

treated = X_scaled[y == 1]
control = X_scaled[y == 0]

nn = NearestNeighbors(n_neighbors=1)
nn.fit(control)

distances, indices = nn.kneighbors(treated)

matched_control = df_esp[y == 0].iloc[indices.flatten()]
matched_treated = df_esp[y == 1].iloc[:len(matched_control)]

matched_df = pd.concat([matched_treated, matched_control])
matched_df.groupby("feature_A_user")[["MAU", "PAU", "PAU_plus", "engagement_depth"]].mean()


## 5. Difference-in-Differences

In [ ]:

did_df = df_esp.copy()
did_df["interaction"] = did_df["feature_A_user"] * did_df["post_launch"]

did_summary = did_df.groupby(
    ["feature_A_user", "post_launch"]
)[["MAU", "PAU_plus"]].mean().reset_index()

did_summary


## 6. Behavioral Funnel

In [ ]:

funnel = df_esp.groupby("feature_A_user")[["MAU", "PAU", "PAU_plus"]].mean()
funnel.T.plot(kind="bar", figsize=(8,5))
plt.title("Behavioral Funnel by Feature A Usage")
plt.ylabel("User Share")
plt.show()


## 7. Conclusion

Feature A is associated with deeper engagement and higher PAU conversion, supporting further experimentation.